# Homework 5 for ORIE 5380
- authors: Hongliang CHI, hc962, email: hc962@cornell.edu

In [1]:
import pandas as pd
from gurobipy import *

### data read

In [2]:
path_data = pd.read_csv('shortest_path_data.txt', delimiter= ' ')
path_data.columns = ['destination','cost']
path_data['origin'] = path_data.index
path_data = path_data[['origin','destination','cost']].reset_index(drop =True)

In [3]:
path_data 

,origin,destination,cost
0,1,2,1.0
1,1,3,2.0
2,2,3,1.0
3,2,4,5.0
4,2,5,2.0
5,3,4,2.0
6,3,5,1.0
7,3,6,4.0
8,4,5,3.0
9,4,6,6.0


### formulation 

Decision variable: Xij ∶ 𝑓𝑙𝑜𝑤 𝑜𝑛 𝑎𝑟𝑐 $ (𝑖, 𝑗)$

Objective function:

min∶ $1.0x12+ 2.0x13+ 1.0x23+ 5.0x24+ 2.0x25+ 2.0x34 + 1.0x35+ 4.0x36$
$+ 3.0x45+ 6.0x46+ 8.0x47+ 3.0x56+ 7.0x57+ 5.0x67+ 2.0x68+ 6.0x78$


Subject To:

$x12 + x13 = 1$

$- x12 + x23 + x24 + x25 = 0$

$- x13 - x23 + x34 + x35 + x36 = 0$

$ - x24 - x34 + x45 + x46 + x47 = 0$

$ - x25 - x35 - x45 + x56 + x57 = 0$

$- x36 - x46 - x56 + x67 + x68 = 0$

$- x47 - x57 - x67 + x78 = 0$

$ - x68 - x78 = -1$

$x12, x13, x23, x24, x25, x34, x35, x36, x45, x46, x47, x56, x57, x67, x68, x78 >=0$

### modeling and solution output

In [5]:
myModel = Model( "shortest_path" )
index_list = [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination)]
coef_list =  [int(i) for i in path_data.cost]
var_dic = {}
constraint = {}

for x in index_list:
    var_dic[x] = myModel.addVar(vtype = GRB.CONTINUOUS , name = "x%s"%str(x))
myModel.update()

objExpr = LinExpr()
for coef, index in zip(coef_list,index_list):
    objExpr += coef * var_dic[index]
myModel.setObjective( objExpr , GRB.MINIMIZE )

Const1 = LinExpr()
Const1 +=  var_dic[12] + var_dic[13]
myModel.addConstr( lhs = Const1 , sense = GRB.EQUAL , \
                   rhs = 1 , name = "1Const" )

Const2 = LinExpr()
Const2 +=  var_dic[23] + var_dic[24] + var_dic[25] - var_dic[12] 
myModel.addConstr( lhs = Const2 , sense = GRB.EQUAL , \
                   rhs = 0 , name = "2Const" )

Const3 = LinExpr()
Const3 +=  var_dic[34] + var_dic[35] + var_dic[36] - var_dic[13] - var_dic[23]
myModel.addConstr( lhs = Const3 , sense = GRB.EQUAL , \
                   rhs = 0 , name = "3Const" )

Const4 = LinExpr()
Const4 +=  var_dic[45] + var_dic[46] + var_dic[47] - var_dic[24] - var_dic[34]
myModel.addConstr( lhs = Const4 , sense = GRB.EQUAL , \
                   rhs = 0 , name = "4Const" )

Const5 = LinExpr()
Const5 +=  var_dic[56] + var_dic[57] - var_dic[25] - var_dic[35] - var_dic[45]
myModel.addConstr( lhs = Const5 , sense = GRB.EQUAL , \
                   rhs = 0 , name = "5Const" )

Const6 = LinExpr() 
Const6 +=   var_dic[67] +  var_dic[68] - var_dic[36] - var_dic[46] - var_dic[56]
myModel.addConstr( lhs = Const6 , sense = GRB.EQUAL , \
                   rhs = 0 , name = "6Const" )

Const7 = LinExpr()
Const7 += var_dic[78]  - var_dic[47] - var_dic[57] - var_dic[67] 
myModel.addConstr( lhs = Const7 , sense = GRB.EQUAL , \
                   rhs = 0 , name = "7Const" )
Const8 = LinExpr()
Const8 += -var_dic[68]  - var_dic[78] 
myModel.addConstr( lhs = Const8 , sense = GRB.EQUAL , \
                   rhs = -1 , name = "8Const" )

myModel.update()
# write the model in a file to make sure it is constructed correctly
myModel.write( filename = "shortest_path.lp" )

# optimize the model
myModel.optimize()

# check the status of the model
curStatus = myModel.status
if curStatus in (GRB.Status.INF_OR_UNBD, GRB.Status.INFEASIBLE, \
              GRB.Status.UNBOUNDED):
    print( "Could not find the optimal solution" )
    exit(1)

# print optimal objective and optimal solution
print( "\nOptimal Objective: " + str( myModel.ObjVal ) )
print( "\nOptimal Solution:" )
myVars = myModel.getVars()
for curVar in myVars:
    print ( curVar.varName + " " + str( curVar.x ) )
    
print( "\nOptimal Dual Solution:" )
myConsts = myModel.getConstrs()
for curConst in myConsts:
    print ( curConst.constrName + " " + str( curConst.pi ) )

Optimize a model with 8 rows, 16 columns and 32 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 14 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9920000e+00   2.004000e+00   0.000000e+00      0s
       3    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds
Optimal objective  8.000000000e+00

Optimal Objective: 8.0

Optimal Solution:
x12 0.0
x13 1.0
x23 0.0
x24 0.0
x25 0.0
x34 0.0
x35 1.0
x36 0.0
x45 0.0
x46 0.0
x47 0.0
x56 1.0
x57 0.0
x67 0.0
x68 1.0
x78 0.0

Optimal Dual Solution:
1Const 3.0
2Const 2.0
3Const 1.0
4Const 0.0
5Const 0.0
6Const -3.0
7Const 1.0
8Const -5.0


In [9]:
myModel = Model( "shortest_path" )
start_node = 1 
end_node = 8 
index_list = [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination)]
coef_list =  [int(i) for i in path_data.cost]
var_dic = {}
constraint = {}

for x in index_list:
    var_dic[x] = myModel.addVar(vtype = GRB.CONTINUOUS , name = "x%s"%str(x))
myModel.update()

objExpr = LinExpr()
for coef, index in zip(coef_list,index_list):
    objExpr += coef * var_dic[index]
myModel.setObjective( objExpr , GRB.MINIMIZE )

Const_start = LinExpr()
for edge in [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination) if i == start_node]:
    Const_start += var_dic[edge]
myModel.addConstr( lhs = Const_start , sense = GRB.EQUAL , \
                   rhs = 1 , name = "Const_start" )

Const_end = LinExpr()
for edge in [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination) if j == end_node]:
    Const_end += -1 * var_dic[edge]
myModel.addConstr( lhs = Const_end , sense = GRB.EQUAL , \
                   rhs = -1 , name = "Const_end" )

Const_end = LinExpr()
for node in list(set(path_data.origin.to_list() + path_data.destination.to_list()) -set([start_node, end_node])):
    constExpr = LinExpr()
    for outbound in [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination) if i == node]:
        constExpr += var_dic[outbound]
    for inbound in [int(str(i) + str(j)) for i ,j in zip(path_data.origin,path_data.destination) if j == node]:
        constExpr += -1* var_dic[inbound]
    myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL , rhs = 0 , \
                       name = "node-" + str( node ) )

myModel.update()
# write the model in a file to make sure it is constructed correctly
myModel.write( filename = "shortest_path.lp" )

# optimize the model
myModel.optimize()

# check the status of the model
curStatus = myModel.status
if curStatus in (GRB.Status.INF_OR_UNBD, GRB.Status.INFEASIBLE, \
              GRB.Status.UNBOUNDED):
    print( "Could not find the optimal solution" )
    exit(1)

# print optimal objective and optimal solution
print( "\nOptimal Objective: " + str( myModel.ObjVal ) )
print( "\nOptimal Solution:" )
myVars = myModel.getVars()
for curVar in myVars:
    print ( curVar.varName + " " + str( curVar.x ) )
    
print( "\nOptimal Dual Solution:" )
myConsts = myModel.getConstrs()
for curConst in myConsts:
    print ( curConst.constrName + " " + str( curConst.pi ) )

Optimize a model with 8 rows, 16 columns and 32 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.04s
Presolved: 6 rows, 14 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9920000e+00   2.004000e+00   0.000000e+00      0s
       3    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.08 seconds
Optimal objective  8.000000000e+00

Optimal Objective: 8.0

Optimal Solution:
x12 0.0
x13 1.0
x23 0.0
x24 0.0
x25 0.0
x34 0.0
x35 1.0
x36 0.0
x45 0.0
x46 0.0
x47 0.0
x56 1.0
x57 0.0
x67 0.0
x68 1.0
x78 0.0

Optimal Dual Solution:
Const_start 3.0
Const_end -5.0
node-2 2.0
node-3 1.0
node-4 0.0
node-5 0.0
node-6 -3.0
node-7 1.0
